In [1]:
import pandas as pd 
import numpy as np
from scipy.spatial.distance import cosine
import string
import re

In [2]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Transformer, Pooling

c:\Users\mypc\anaconda3\envs\SymanticAnalysis\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\mypc\anaconda3\envs\SymanticAnalysis\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
transformer = Transformer("w601sxs/b1ade-embed")
pooling = Pooling(transformer.get_word_embedding_dimension(), pooling_mode="mean") 
model = SentenceTransformer(modules=[transformer, pooling])

c:\Users\mypc\anaconda3\envs\SymanticAnalysis\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mypc\.cache\huggingface\hub\models--w601sxs--b1ade-embed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [24]:
df = pd.read_csv("D:\AIO\STT_Model_Evaluation\Transcription Results\parakeet-ctc-1.1b_Results.csv")

In [25]:
df.head()

,audio_path,ground_truth,transcription,inference_time
0,2024_07_09 10_31_30 NtuyGQ.wav,I would like to order a Patty Melt without the...,i would like to order a patty melt without the...,0.582718
1,2024_07_09 21_11_19 3yCioQ.wav,May I get a small California salad with arugul...,may i get a small california salad with arugul...,0.065769
2,2024_07_12 03_34_50 Ap-fWA.wav,"""May I order the House Special Beef, and could...",may i order a house special beef and could you...,0.062657
3,2024_07_24 12_22_36 fNFdZw.wav,Can I have a Buffalo Chicken Sandwich with a s...,can i have the buffalo chicken sandwich with a...,0.062813
4,2024_07_10 09_55_57 x5sltA.wav,"""I would like to order a Hawaiian Pizza with P...",i would like to order a hawaiian pizza with pa...,0.056818


In [26]:
df['Semantic_Analysis'] = 0.0

In [27]:
for index, row in df.iterrows():
    ground_truth = row["ground_truth"]
    after_finetuning_transcription = row["transcription"]
    
    
    ground_truth = ground_truth.strip().lower().strip(string.punctuation)
    after_finetuning_transcription.strip().lower().strip(string.punctuation)
    
    ground_truth = re.sub(r'[^\w\s]', '', ground_truth)
    after_finetuning_transcription = re.sub(r'[^\w\s]', '', after_finetuning_transcription)
    

    sentences_1 = [ground_truth]
    sentences_2 = [after_finetuning_transcription]
    
    embeddings1 = model.encode(sentences_1)
    embeddings2 = model.encode(sentences_2)
    
    # Calculate cosine similarities
    similarities = []
    for emb1 in embeddings1:
        similarities.append([1 - cosine(emb1, emb2) for emb2 in embeddings2])
    
    # Assign the similarity scores to the new columns
    df.at[index, 'Semantic_Analysis'] = similarities[0][0]


In [28]:
df.to_csv("D:\AIO\STT_Model_Evaluation\Transcription Results\parakeet-ctc-1.1b_Results.csv")